## Part 4: Fraud

In [1]:
%run "../scripts/ETL.py"

#### Consumer fraud data
The consumer fraud data shows the list of transactions with the probability that the entire batch of transaction done by that consumer on that day could be considered fraudulent.

In [2]:
print(fraud_consumer.count())
fraud_consumer.limit(5)

34864


user_id,order_datetime,fraud_probability
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


#### Merchant fraud data
The merchant fraud data shows the probability that the entire batch of transaction done with that merchant on that day could be considered fraudulent.
This could happen when merchants are targeted by scalper bots or other attacks during sales (i.e. the consumers committed fraud for merchants whether on purpose or because their details were stolen).

In [3]:
print(fraud_merchants.count())
fraud_merchants.limit(5)

114


merchant_abn,order_datetime,fraud_probability
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


#### Dataset to join fraud data

In [4]:
final_join.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Ashley Harris,84610 Amy Harbors,QLD,4184,Female,12516851436,285.1622236961745,eda709b9-dcd3-449...,2022-02-08,14971,1144223
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,William Ali,3166 Annette Port...,NSW,1022,Male,12516851436,75.4722398570927,92f8ad85-c6d8-409...,2022-08-13,20642,1153786
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Anna Strickland,906 Pham Village ...,SA,5250,Female,12516851436,47.830718899387165,ab3d0252-d05f-44f...,2022-04-10,20311,1223293
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Michael Bishop,98598 Murray Trai...,QLD,4580,Male,12516851436,259.43552349322687,0aad2674-03ee-4d8...,2022-03-25,22971,443950
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Mary Diaz,4111 Jason Ville,SA,5038,Female,12516851436,62.60867338695297,dfb805b6-1aad-4ea...,2022-08-04,10575,626154


#### Join Fraud dataset to the processed transaction dataset

- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [5]:
# adding consumer fraud data
fj_with_fraud = final_join.join(fraud_consumer, (final_join.user_id == fraud_consumer.user_id) \
                               & (final_join.order_datetime == fraud_consumer.order_datetime), "left_outer")
fj_with_fraud.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,postcode,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,user_id,order_datetime,fraud_probability
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Ashley Harris,84610 Amy Harbors,QLD,4184,Female,12516851436,285.1622236961745,eda709b9-dcd3-449...,2022-02-08,14971,1144223,null,null,null
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,William Ali,3166 Annette Port...,NSW,1022,Male,12516851436,75.4722398570927,92f8ad85-c6d8-409...,2022-08-13,20642,1153786,null,null,null
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Anna Strickland,906 Pham Village ...,SA,5250,Female,12516851436,47.830718899387165,ab3d0252-d05f-44f...,2022-04-10,20311,1223293,null,null,null
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Michael Bishop,98598 Murray Trai...,QLD,4580,Male,12516851436,259.43552349322687,0aad2674-03ee-4d8...,2022-03-25,22971,443950,null,null,null
Mollis Corp.,12516851436,"watch, clock, and...",6.71,a,Mary Diaz,4111 Jason Ville,SA,5038,Female,12516851436,62.60867338695297,dfb805b6-1aad-4ea...,2022-08-04,10575,626154,null,null,null


In [ ]:
# adding merchant fraud data
fj_with_fraud = fj_with_fraud.join(fraud_merchants, (fj_with_fraud.merchant_abn == fraud_merchants.merchant_abn) \
                               & (fj_with_fraud.order_datetime == fraud_merchants.order_datetime), "left_outer")
fj_with_fraud.limit(5)

In [22]:
fj_rem_fraud = fj_with_fraud.filter("fraud_probability is NULL")
fj_rem_fraud.count()

14115157

In [16]:
user_details.count()

499999

In [23]:
final_join.count()

14195505

In [24]:
tbl_merchants.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels
Felis Limited,10023283211,"furniture, home f...",0.18,e
Arcu Ac Orci Corp...,10142254217,"cable, satellite,...",4.22,b
Nunc Sed Company,10165489824,"jewelry, watch, c...",4.4,b
Ultricies Digniss...,10187291046,"wAtch, clock, and...",3.29,b
Enim Condimentum PC,10192359162,music shops - mus...,6.33,a
